# Study Colors of isolated objects


ColorIsolatedObjects.ipynb:
The goal is to generate a non-buggy list of colored objects in order to have the correct color distribution.
The original list **list_of_isolatedobjects.csv** produced by **MultiColor_lightCurves-DMRehearsal2024_01-AuxTel.ipynb** had a bug.

Now this notebook produce the file to use **isolated_objects_magandcolors_psfap_good.csv** from the original source files **sources_objectTable-t3864-multibands-o(.*)-LATISS_runs_AUXTEL_DRP_IMAGING_20230509_20240513_w_2024_20_PREOPS-5146.csv**
in folder **data/photom/sources_objectTable**.

- author : Sylvie Dagoret-Campagne
- creation : 2024-08-01
- update : 2024-08-02 : number id of object + add (ra,dec)
- last update : 2024-09-05 : add comment
- last update : 2024-09-08 : add number of object

In [ ]:
import os
import astropy.units as u
import numpy as np 
import pandas as pd
pd.options.mode.chained_assignment = None
pd.set_option("display.max_columns", None)
from astropy.time import Time

import scipy.stats

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm,SymLogNorm
from matplotlib.patches import Circle,Annulus


from astropy.stats import SigmaClip

In [ ]:
from astropy.io import fits
from astropy.wcs import WCS


import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option('display.max_rows', 100)

import matplotlib.ticker                         # here's where the formatter is
import os
import re
import pandas as pd
import pickle
from collections import OrderedDict

plt.rcParams["figure.figsize"] = (4,3)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'

import scipy
from scipy.optimize import curve_fit

In [ ]:
def convert_fluxtomag(x) :
    """
    The object and source catalogs store only fluxes. There are hundreds of flux-related columns, 
    and to store them also as magnitudes would be redundant, and a waste of space.
    All flux units are nanojanskys. The AB Magnitudes Wikipedia page provides a concise resource 
    for users unfamiliar with AB magnitudes and jansky fluxes. To convert to AB magnitudes use:
    As demonstrated in Section 2.3.2, to add columns of magnitudes after retrieving columns of flux, users can do this:
    results_table['r_calibMag'] = -2.50 * numpy.log10(results_table['r_calibFlux']) + 31.4
    results_table['r_cModelMag'] = -2.50 * numpy.log10(results_table['r_cModelFlux']) + 31.4
    (from DP0 tutorial)
    """
    return -2.50 * np.log10(x) + 31.4

In [ ]:
def convert_fluxtomagerr(row) :
    """
    The object and source catalogs store only fluxes. There are hundreds of flux-related columns, 
    and to store them also as magnitudes would be redundant, and a waste of space.
    All flux units are nanojanskys. The AB Magnitudes Wikipedia page provides a concise resource 
    for users unfamiliar with AB magnitudes and jansky fluxes. To convert to AB magnitudes use:
    As demonstrated in Section 2.3.2, to add columns of magnitudes after retrieving columns of flux, users can do this:
    results_table['r_calibMag'] = -2.50 * numpy.log10(results_table['r_calibFlux']) + 31.4
    results_table['r_cModelMag'] = -2.50 * numpy.log10(results_table['r_cModelFlux']) + 31.4
    (from DP0 tutorial)
    """
    x= row["apFlux_35_0_instFlux"]
    dx = row["apFlux_35_0_instFluxErr"]
    mag =  -2.50 * np.log10(x) + 31.4
    emag = 2.5/np.log(10)*dx/x
    return mag,emag

In [ ]:
def get_df_perband(df):
    dfg = df[df.band == "g"]
    dfr = df[df.band == "r"]
    dfi = df[df.band == "i"]
    dfz = df[df.band == "z"]
    dfy = df[df.band == "y"]
    return (dfg,dfr,dfi,dfz,dfy)

In [ ]:
def get_df_median(df,ctag="mag"):
    dfg,dfr,dfi,dfz,dfy = get_df_perband(df)
    medg = dfg[ctag].median()
    medr = dfr[ctag].median()
    medi = dfi[ctag].median()
    medz = dfz[ctag].median()
    medy = dfy[ctag].median()
    return (medg,medr,medi,medz,medy)    

In [ ]:
def get_df_sigma(df,ctag="mag"):
    dfg,dfr,dfi,dfz,dfy = get_df_perband(df)   
    (medg,medr,medi,medz,medy) = get_df_median(df,ctag)
    
    #dfg["sigma"] = (dfg[tag]- medg)**2
    dfg.loc[:,"sigma"] = dfg.apply( lambda row : (row[ctag]- medg)**2,axis=1,result_type="expand")
    dfr["sigma"] = dfr.apply( lambda row : (row[ctag]- medr)**2,axis=1,result_type="expand")
    if len(dfi)>0:
        flag_dfi = True
        dfi["sigma"] = dfi.apply( lambda row : (row[ctag]- medi)**2,axis=1,result_type="expand")
    else:
        flag_dfi = False
    dfz["sigma"] = dfz.apply( lambda row : (row[ctag]- medz)**2,axis=1,result_type="expand")
    dfy["sigma"] = dfy.apply( lambda row : (row[ctag]- medy)**2,axis=1,result_type="expand")
    
    sigg = np.sqrt(dfg["sigma"].mean())
    sigr = np.sqrt(dfr["sigma"].mean())
    if flag_dfi:
        sigi = np.sqrt(dfi["sigma"].mean())
    else:
        sigi = 0
    sigz = np.sqrt(dfz["sigma"].mean())
    sigy = np.sqrt(dfy["sigma"].mean())
    return (sigg,sigr,sigi,sigz,sigy)

    

In [ ]:
def plothistogram_magperband(df):
    objectnum = df["obj_index"].unique()[0]
    dfg,dfr,dfi,dfz,dfy = get_df_perband(df)

    (medg,medr,medi,medz,medy) = get_df_median(df)
    (sigg,sigr,sigi,sigz,sigy) = get_df_sigma(df)
    
    fig,axs = plt.subplots(1,5,figsize=(16,3))
    axes = axs.flatten()
    
    label = f"$\sigma = $ {sigg*1000:.2f} mmag"
    dfg["mag"].plot.hist(ax=axes[0],bins=20,facecolor="g",xlabel="mag")
    axes[0].axvline(medg,color="b",lw=3)
    axes[0].set_title(label)

    label = f"$\sigma = $ {sigr*1000:.2f} mmag"
    dfr["mag"].plot.hist(ax=axes[1],bins=20,facecolor="r",xlabel="mag")
    axes[1].axvline(medr,color="b",lw=3)
    axes[1].set_title(label)

    label = f"$\sigma = $ {sigi*1000:.2f} mmag"
    dfi["mag"].plot.hist(ax=axes[2],bins=20,facecolor="y",xlabel="mag")
    axes[2].axvline(medi,color="b",lw=3)
    axes[2].set_title(label)

    label = f"$\sigma = $ {sigz*1000:.2f} mmag"
    dfz["mag"].plot.hist(ax=axes[3],bins=20,facecolor="grey",xlabel="mag")
    axes[3].axvline(medz,color="b",lw=3)
    axes[3].set_title(label)

    label = f"$\sigma = $ {sigy*1000:.2f} mmag"
    dfy["mag"].plot.hist(ax=axes[4],bins=20,facecolor="k",xlabel="mag")
    axes[4].axvline(medy,color="b",lw=3)
    axes[4].set_title(label)

    title = f"object {objectnum}"
    plt.suptitle(title,fontsize=14,fontweight="bold")
    plt.tight_layout()
    plt.show()
       

In [ ]:
def plothistogram_valperband(df,ctag):
    objectnum = df["obj_index"].unique()[0]
    dfg,dfr,dfi,dfz,dfy = get_df_perband(df)

    (medg,medr,medi,medz,medy) = get_df_median(df,ctag)
    (sigg,sigr,sigi,sigz,sigy) = get_df_sigma(df,ctag)
    
    fig,axs = plt.subplots(1,5,figsize=(16,3))
    axes = axs.flatten()
    
    label = f"$\sigma = $ {sigg*1000:.2f} mmag"
    dfg[ctag].plot.hist(ax=axes[0],bins=20,facecolor="g",xlabel="mag")
    axes[0].axvline(medg,color="b",lw=3)
    axes[0].set_title(label)

    label = f"$\sigma = $ {sigr*1000:.2f} mmag"
    dfr[ctag].plot.hist(ax=axes[1],bins=20,facecolor="r",xlabel="mag")
    axes[1].axvline(medr,color="b",lw=3)
    axes[1].set_title(label)

    label = f"$\sigma = $ {sigi*1000:.2f} mmag"
    dfi[ctag].plot.hist(ax=axes[2],bins=20,facecolor="y",xlabel="mag")
    axes[2].axvline(medi,color="b",lw=3)
    axes[2].set_title(label)

    label = f"$\sigma = $ {sigz*1000:.2f} mmag"
    dfz[ctag].plot.hist(ax=axes[3],bins=20,facecolor="grey",xlabel="mag")
    axes[3].axvline(medz,color="b",lw=3)
    axes[3].set_title(label)

    label = f"$\sigma = $ {sigy*1000:.2f} mmag"
    dfy[ctag].plot.hist(ax=axes[4],bins=20,facecolor="k",xlabel="mag")
    axes[4].axvline(medy,color="b",lw=3)
    axes[4].set_title(label)

    title = f"object {objectnum}"
    plt.suptitle(title,fontsize=14,fontweight="bold")
    plt.tight_layout()
    plt.show()
       

## Config

In [ ]:
path = "data/photom/sources_objectTable"

## Find input files

In [ ]:
list_of_files = os.listdir(path) 
list_of_files = sorted(list_of_files)

In [ ]:
tag = "^sources_objectTable-t3864-multibands-o(.*)-LATISS_runs_AUXTEL_DRP_IMAGING_20230509_20240513_w_2024_20_PREOPS-5146.csv$"
selected_files = []
selected_objectnum = []
for filename in list_of_files: 
    result = re.findall(tag,filename)
    if len(result)>0:
        num = int(result[0])
        selected_objectnum.append(num)
        selected_files.append(filename) 
        
selected_objectnum = np.array(selected_objectnum)
selected_files = np.array(selected_files)

In [ ]:
index_sorted = np.argsort(selected_objectnum)
selected_objectnum = selected_objectnum[index_sorted] 
selected_files = selected_files[index_sorted]
N = len(selected_files)

## Read pandas dataframe

In [ ]:
all_df = []
for index,filename in enumerate(selected_files):
    fullfilename = os.path.join(path,filename)
    df = pd.read_csv(fullfilename,index_col=0)
    # calculate the magnitude I want
    df[["mag","emag"]] = df.apply(lambda row : convert_fluxtomagerr(row), axis=1,result_type = 'expand')
    all_df.append(df)

## Study magnitudes

In [ ]:
index = 0
objectnum = selected_objectnum[index] 

In [ ]:
the_df = all_df[index]

In [ ]:
the_df.head()

In [ ]:
plothistogram_magperband(the_df)

In [ ]:
plothistogram_valperband(the_df,ctag="psfMag")

In [ ]:
dfg,dfr,dfi,dfz,dfy = get_df_perband(the_df)   
(medg,medr,medi,medz,medy) = get_df_median(the_df)
(sigg,sigr,sigi,sigz,sigy) = get_df_sigma(the_df)

## Create pandas dataframe on aperture photometry isolated object data

In [ ]:
df_col_ap = pd.DataFrame(columns=["obj","ra","dec","g","r","i","z","y","sg","sr","si","sz","sy"])
for index in range(N):
    the_df = all_df[index]
    ra = the_df['ra'].mean()
    dec = the_df['dec'].mean()
    obj=selected_objectnum[index]
    plothistogram_magperband(the_df)

    (medg,medr,medi,medz,medy) = get_df_median(the_df)
    (sigg,sigr,sigi,sigz,sigy) = get_df_sigma(the_df)
    df_col_ap.loc[index]=[obj,ra,dec,medg,medr,medi,medz,medy,sigg,sigr,sigi,sigz,sigy]

In [ ]:
ramin = df_col_ap["ra"].min()
ramax = df_col_ap["ra"].max()
decmin = df_col_ap["dec"].min()
decmax = df_col_ap["dec"].max()
dra = (ramax-ramin)*60.
ddec = (decmax-decmin)*60

In [ ]:
props = dict(boxstyle='round', facecolor='white', alpha=0.5)
#textstr = '\n'.join((
#    r'$\mu=%.2f$' % (mu, ),
#    r'$\mathrm{median}=%.2f$' % (median, ),
#    r'$\sigma=%.2f$' % (sigma, )))
textstr = f"FOV = {dra:.2f} arcmin x {ddec:.2f} arcmin"

In [ ]:
df_col_ap["g_r"] = df_col_ap["g"]- df_col_ap["r"]
df_col_ap["z_y"] = df_col_ap["z"]- df_col_ap["y"]
df_col_ap["sig_g_r"] = df_col_ap.apply(lambda row: np.sqrt(row["sg"]**2+ row["sr"]**2),axis=1)
df_col_ap["sig_z_y"] = df_col_ap.apply(lambda row: np.sqrt(row["sz"]**2+ row["sy"]**2),axis=1)

In [ ]:
#df_col_ap

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(16,16))
ax.errorbar(df_col_ap["z_y"],df_col_ap["g_r"],xerr=df_col_ap["sig_z_y"] , yerr=df_col_ap["sig_g_r"], fmt=".",color='k',ecolor='k',markersize=1) 
df_col_ap.plot.scatter(x="z_y", y="g_r",marker= "o", s=80,c="sy",cmap="jet",ax=ax,colorbar=False)
ax.set_xlabel("Z-Y (mag)")
ax.set_ylabel("G-R (mag)")
ax.set_title("color-color on aperture photometry")
ax.set_aspect("equal")
ax.grid()


for index,row in df_col_ap.iterrows():
    idobj = int(row.obj)
    txtlabel = f"{idobj}({index})"
    x = row["z_y"]
    y = row["g_r"]
    if index%2 == 0:
        ax.text(x-0.005,y+.005,txtlabel,size= 'xx-large',color="r")
    else:
        ax.text(x+0.005,y-0.005,txtlabel,size= 'xx-large',color="r")


plt.show()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(15,15))
df_col_ap.plot.scatter(x="ra", y="dec",marker= "o", s=100,c="z_y",cmap="jet",ax=ax,colorbar=False)
ax.set_xlabel("ra (degree)")
ax.set_ylabel("dec (mag)")
ax.set_title("Isolated object")
ax.set_aspect("equal")
ax.grid()

cbar = plt.colorbar(ax.collections[0],ax=ax,orientation = 'horizontal',shrink=1.0)
cbar.ax.set_xlabel('Z-Y (mag) (aperture photometry)', rotation=0)

for index,row in df_col_ap.iterrows():
    idobj = int(row.obj)
    x = row["ra"]
    y = row["dec"]
    if index%2 == 0:
        ax.text(x-0.002,y+.002,str(idobj),size= 'x-large')
    else:
        ax.text(x+0.002,y-0.002,str(idobj),size= 'x-large')


ax.text(0.01, 0.99, textstr, transform=ax.transAxes, fontsize=14,
        verticalalignment='top', bbox=props)
plt.tight_layout()
plt.show()

## Create pandas dataframe on psf photometry isolated object data

In [ ]:
df_col_psf = pd.DataFrame(columns=["obj","ra","dec","g","r","i","z","y","sg","sr","si","sz","sy"])
for index in range(N):
    the_df = all_df[index]
    ra = the_df['ra'].mean()
    dec = the_df['dec'].mean()
    obj=selected_objectnum[index]
    
    plothistogram_valperband(the_df,ctag="psfMag")

    (medg,medr,medi,medz,medy) = get_df_median(the_df,ctag="psfMag")
    (sigg,sigr,sigi,sigz,sigy) = get_df_sigma(the_df,ctag="psfMag")
    df_col_psf.loc[index]=[obj,ra,dec,medg,medr,medi,medz,medy,sigg,sigr,sigi,sigz,sigy]

In [ ]:
df_col_psf["g_r"] = df_col_psf["g"]- df_col_psf["r"]
df_col_psf["z_y"] = df_col_psf["z"]- df_col_psf["y"]
df_col_psf["sig_g_r"] = df_col_psf.apply(lambda row: np.sqrt(row["sg"]**2+ row["sr"]**2),axis=1)
df_col_psf["sig_z_y"] = df_col_psf.apply(lambda row: np.sqrt(row["sz"]**2+ row["sy"]**2),axis=1)

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(16,16))
ax.errorbar(df_col_psf["z_y"],df_col_psf["g_r"],xerr=df_col_psf["sig_z_y"] , yerr=df_col_psf["sig_g_r"], fmt=".",color='k',ecolor='k',markersize=1) 
df_col_psf.plot.scatter(x="z_y", y="g_r",marker= "o", s=80,c="sy",cmap="jet",ax=ax,colorbar=False)
ax.set_xlabel("Z-Y mag")
ax.set_ylabel("G-R mag")
ax.set_title("color-color on psf photometry")
ax.set_aspect("equal")
ax.grid()

for index,row in df_col_psf.iterrows():
    idobj = int(row.obj)
    txtlabel = f"{idobj}({index})"
    x = row["z_y"]
    y = row["g_r"]
    if index%2 == 0:
        ax.text(x-0.005,y+.005,txtlabel,size= 'xx-large',color="r")
    else:
        ax.text(x+0.005,y-0.005,txtlabel,size= 'xx-large',color="r")


plt.show()

In [ ]:
df_col_psf

In [ ]:
df_col_psf["sizecode"] = (15.5-df_col_psf["z"])*50

In [ ]:
N

In [ ]:
flag_show_numbers = False

fig,ax = plt.subplots(1,1,figsize=(10,12))
df_col_psf.plot.scatter(x="ra", y="dec",marker= "o", s="sizecode",c="z_y",cmap="jet",ax=ax,colorbar=False)
ax.set_xlabel("ra (degree)")
ax.set_ylabel("dec (mag)")
ax.set_title("Isolated object")
ax.set_aspect("equal")
ax.grid()

cbar = plt.colorbar(ax.collections[0],ax=ax,orientation = 'horizontal',shrink=1.0,pad=0.08)
cbar.ax.set_xlabel('Z-Y (mag)(calib AB psf photometry)', rotation=0)

if flag_show_numbers:
    for index,row in df_col_psf.iterrows():
        idobj = int(row.obj)
        x = row["ra"]
        y = row["dec"]
        if index%2 == 0:
            ax.text(x-0.002,y+.002,str(idobj),size= 'x-large')
        else:
            ax.text(x+0.002,y-0.002,str(idobj),size= 'x-large')

ax.text(0.01, 0.99, textstr, transform=ax.transAxes, fontsize=14,verticalalignment='top', bbox=props)
ax.text(0.85, 0.99, f"Nobj = {N}", transform=ax.transAxes, fontsize=14,verticalalignment='top', bbox=props)
plt.tight_layout()
plt.show()

## Save file

In [ ]:
df_obj_colors = pd.merge(left=df_col_psf,right=df_col_ap,on="obj",suffixes=("_psf","_ap"))

In [ ]:
df_obj_colors.to_csv("isolated_objects_magandcolors_psfap_good.csv")